<a href="https://colab.research.google.com/github/Zack1243/Novel-Webscraper/blob/multi-threaded-downloads/Webscraper3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from bs4 import BeautifulSoup as b4
import asyncio
from google.colab import files
import multiprocessing
import os
import requests
import time
import zipfile
!pip install selenium

In [20]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [21]:
TRIAL = 3
NUM_THREADS = 4
LINKS = ["https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-1",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-2",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-3",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-4",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-5",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-6",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-7",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-8"]

## Instantiate browser

In [22]:
def getBrowser():
  options = webdriver.FirefoxOptions()
  options.add_argument('--headless')
  browser = webdriver.Firefox(options=options)
  return browser

### Parse chapter

In [23]:
async def parseFile(html):

  # Start beautiful soup object and parse
  soup = b4(html, features="html.parser")
  tag = soup.article.find_all('span')

  # Get title
  for t in tag:
    if "Chapter" in t.get_text():
      title = t.get_text() + ".html"

  # Get chapter content
  soup = soup.find_all('p')
  with open(title, "w" , encoding='utf-8') as file:
    for paragraph in soup:
        file.write(str(paragraph) + "\n")  # Write each paragraph to the file
    print(str(soup))
  file.close()

### Download main webpage

In [34]:
async def getChapter(browser, url, trial, thread_id):
  print(f"Thread {thread_id} attempting to download url: {url}")
  # while True:
  try:
    browser.get(url)
    timeout_in_seconds = 10
    WebDriverWait(browser, timeout_in_seconds).until(EC.presence_of_element_located((By.ID, 'chapter-container')))
    await parseFile(browser.page_source)
  except TimeoutException:
    if trial >= TRIAL:
      with open(url, "w") as file:
        file.write("I DONE GOOFED!!!")
      file.close()
      print("I give up...")
    else:
      await asyncio.sleep(0.1)
      print(f"Thread {thread_id} Download failed trial {trial}")
      trial += 1
      await getChapter(browser, url, trial, thread_id)

## Get Chapter page links

In [35]:
async def getChapterPage(queue, browser, url, trial, thread_id):
  try:
    # Get webpage
    print(f"Thread {thread_id} getChapterPages trying url: {url}...")
    browser.get(url)

    # Start beautiful soup html parsing
    soup = b4(browser.page_source, features="html.parser")
    links = soup.find(class_="pagenav").find_all(class_="PagedList-skipToNext")

    # Append chapter and chapter page links
    if links:
      queue.put_nowait("https://www.lightnovelpub.com" + links[0].find('a')['href'])
    strainer = soup.find(class_="chapter-list").find_all('a')
    for link in strainer:
      queue.put_nowait("https://www.lightnovelpub.com"+ link['href'])


  # Try to get webpage a total of TRIAL times before giving up
  except TimeoutException:
    if trial >= TRIAL:
      with open(url, "w") as file:
        file.write("I DONE GOOFED!!!")
      file.close()
      print("I give up...")
    else:
      await asyncio.sleep(0.1)
      print(f"Thread {thread_id} Download failed trial {trial}")
      await getChapterPage(queue, browser, url, trial + 1, thread_id)

### Get links and go to next chapter page, if avaliable (testing from file)

In [26]:
async def getChapterPages(queue, browser, thread_id):
  while True:

    # Fet item from queue
    url = await queue.get()

    await asyncio.sleep(0.1)

    # If this is a chapter page
    if "page" in url:
      await getChapterPage(queue, browser, url, 0, thread_id)

    # Else, this url is a chapter
    else:
      await getChapter(browser, url, 0, thread_id)
    #print(f"Finished downloading/adding from url: {url}")
    # Mark as done
    queue.task_done()


## Get the first webpage

In [27]:
#TODO
  # Add the link to list of failed downloads if failed
def getFirstPage(url, browser, trial):
  try:
    queue = asyncio.Queue()

    # Get and parse webpage
    browser.get(url)
    soup = b4(browser.page_source, features="html.parser")

    # Get chapter links and chapter page links
    #TODO: Error check if the next page does not exist
    links = soup.find(class_="pagenav").find_all(class_="PagedList-skipToNext")
    strainer = soup.find(class_="chapter-list").find_all('a')

    # Found both chapter page links and chapter links
    if links:
      # Queue the next chapter page url
      url1 = "https://www.lightnovelpub.com" + links[0].find('a')['href']
      #print(f"Adding url to queue: {url1}")
      queue.put_nowait(url1)
      for link in strainer:
        #print(f"Adding url to queue: {link}")
        queue.put_nowait("https://www.lightnovelpub.com"+ link['href'])
      return queue

    # Could only find chapter links
    elif strainer:
      for link in strainer:
        queue.put_nowait("https://www.lightnovelpub.com"+ link['href'])
      return queue

    # Got nothing
    else:
      return -1

  # Failed to download webpage
  except TimeoutException:
    # Give up!
    if trial >= TRIAL:
      # Write a file saying what failed
      print(f"Failed to find/get input url: {url}")
      return -1

    # Try again, up to 3 times
    else:
      print(f"Download failed trial {trial}")
      getFirstPage(url, browser, trial + 1)

## Async Download Chapter pages

In [28]:
async def asyncDownload(url, browser):
  NUM_THREADS = 4
  queue = getFirstPage(url, browser, 0)
  browser.quit()
  print(queue)
  if queue == -1:
    return

  # Adjust threads to queue size
  if queue.qsize() < NUM_THREADS:
    NUM_THREADS = queue.qsize()

  # Instantiate task list and browsers
  tasks = []
  browsers = []
  for i in range(NUM_THREADS):
    browsers.append(getBrowser())
    browsers.append(i)
    print(f"Starting browser {i}")
    task = asyncio.create_task(getChapterPages(queue, browsers[i], i))
    tasks.append(task)

  # Wait for all queue items to be downloaded
  await queue.join()

  # Quit all browsers
  for browser in browsers:
    browser.quit()

  # Cancel all tasks
  for task in tasks:
    task.cancel()

  # Wait for tasks to be canceled
  await asyncio.gather(*tasks, return_exceptions=True)




  # Error checking
    # Check at the last chapter page
    # Check if there is a next chapter page
    # List of failed downloads

## Zip up files to be downloaded

In [29]:
def downloadZip(title):
  directory = os.getcwd()

  # List all files in the directory
  all_files = os.listdir(directory)

  # Specify the name of the zip file to create
  zip_file_name = title + ".zip"

  # Create a zip file
  with zipfile.ZipFile(zip_file_name, 'w') as zipf:
      # Add each file in the directory to the zip file
      for file in all_files:
          file_path = os.path.join(directory, file)
          zipf.write(file_path, os.path.basename(file_path))

  print("Zip file created:", zip_file_name)

  files.download(directory + '/' + zip_file_name)

## Make and change to directory (book's title)

In [30]:
def getDir(directory):
  if not os.path.exists(directory):
    os.mkdir(directory)
    os.chdir(directory)
  elif os.getcwd != directory:
    os.chdir(directory)
  else:
    pass

In [31]:
async def main1(url, title):
    # Change directory
    getDir(title)

    # Start async downloading chapter pages
    await asyncDownload(url, getBrowser())

    # Zip and download chapters
    downloadZip(title)

### Input url

In [36]:
# Ask user for ln information
#url = input("Enter URL: ")
#title = input("Enter directory: ")
url = "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities/chapters"
title = "advant-of-the-three-calamities"
await main1(url, title)

<Queue maxsize=0 _queue=['https://www.lightnovelpub.com/novel/advent-of-the-three-calamities/chapters?page=2', 'https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-1', 'https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-2', 'https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-3', 'https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-4', 'https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-5', 'https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-6', 'https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-7', 'https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-8', 'https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-9', 'https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-10', 'https://www.lightnovelpub.com/novel/a

CancelledError: 

In [33]:
import asyncio
import random
import time



async def sleep(name, sleep_for):
    await asyncio.sleep(sleep_for)
    print(f'{name} has slept for {sleep_for:.2f} seconds')




async def worker(name, queue):
    while True:
        # Get a "work item" out of the queue.
        sleep_for = await queue.get()

        # Sleep for the "sleep_for" seconds.
        await sleep(name, sleep_for)

        # Notify the queue that the "work item" has been processed.
        queue.task_done()

        print(f'{name} has slept for {sleep_for:.2f} seconds')


async def main():
    # Create a queue that we will use to store our "workload".
    queue = asyncio.Queue()

    # Generate random timings and put them into the queue.
    total_sleep_time = 0
    for _ in range(20):
        sleep_for = random.uniform(0.05, 1.0)
        total_sleep_time += sleep_for
        queue.put_nowait(sleep_for)

    # Create three worker tasks to process the queue concurrently.
    tasks = []
    for i in range(3):
        task = asyncio.create_task(worker(f'worker-{i}', queue))
        tasks.append(task)

    # Wait until the queue is fully processed.
    started_at = time.monotonic()
    await queue.join()
    total_slept_for = time.monotonic() - started_at

    # Cancel our worker tasks.
    for task in tasks:
        task.cancel()
    # Wait until all worker tasks are cancelled.
    await asyncio.gather(*tasks, return_exceptions=True)

    print('====')
    print(f'3 workers slept in parallel for {total_slept_for:.2f} seconds')
    print(f'total expected sleep time: {total_sleep_time:.2f} seconds')


await main()

worker-2 has slept for 0.49 seconds
worker-2 has slept for 0.49 seconds
worker-0 has slept for 0.61 seconds
worker-0 has slept for 0.61 seconds
worker-1 has slept for 0.61 seconds
worker-1 has slept for 0.61 seconds
worker-1 has slept for 0.22 seconds
worker-1 has slept for 0.22 seconds
worker-0 has slept for 0.54 seconds
worker-0 has slept for 0.54 seconds
worker-2 has slept for 0.70 seconds
worker-2 has slept for 0.70 seconds
worker-0 has slept for 0.55 seconds
worker-0 has slept for 0.55 seconds
worker-1 has slept for 0.94 seconds
worker-1 has slept for 0.94 seconds
worker-2 has slept for 0.67 seconds
worker-2 has slept for 0.67 seconds
worker-2 has slept for 0.09 seconds
worker-2 has slept for 0.09 seconds
worker-0 has slept for 0.61 seconds
worker-0 has slept for 0.61 seconds
worker-1 has slept for 0.74 seconds
worker-1 has slept for 0.74 seconds
worker-0 has slept for 0.25 seconds
worker-0 has slept for 0.25 seconds
worker-2 has slept for 0.62 seconds
worker-2 has slept for 0.62 